# Client Project 5: EDA part

In [2]:
# importing Necessary Library
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import datetime
import json
import os
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.externals.six import StringIO 
from IPython.display import Image  
from sklearn.tree import export_graphviz
#import pydotplus



#  Configure Visual setting 
sns.set_style('whitegrid')
#plt.style.use(['bmh','presentation'])
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.style.use('ggplot')
np.random.seed(42)
sns.set()
%config InlineBackend.figure_format = 'retina'
%matplotlib inline    

#### 1 Load the Data

#### Load in the the data of scraped results

In [5]:
# Lets load the Concatenated DataFrame and the engine specification will help to delimit the outliers
df1 = pd.read_csv('./data/ne bomb cyclone data w:o location/ne_bomb_cyclone_before_tweets.csv',index_col ='Unnamed: 0' , engine='python')
df2 = pd.read_csv('./data/ne bomb cyclone data w:o location/ne_bomb_cyclone_during_tweets.csv',index_col ='Unnamed: 0' , engine='python')
df3 = pd.read_csv('./data/ne bomb cyclone data w:o location/ne_bomb_cyclone_after_tweets.csv',index_col ='Unnamed: 0' , engine='python')


# To concatenate the three dataframes
frames = [df1,df2,df3]

df = pd.concat(frames)
#df = pd.read_csv('Project3edapartone.csv',index_col ='Unnamed: 0' , engine='python')

In [6]:
df.head()

,Event,Stage,Query Date,Query Term,Id,Username,Text,Date,Hashtags,Location
0,ne_bomb_cyclone,before,2019-10-15,power outage,1183894397492043777,meekers999,Let's lose Govenor Gruesome in Cali please. He...,2019-10-14 23:56:34+00:00,NaN,NaN
1,ne_bomb_cyclone,before,2019-10-15,power outage,1183894362725289984,sharethiscrime,"Last time I checked, he's still a floofy baby ...",2019-10-14 23:56:26+00:00,NaN,NaN
2,ne_bomb_cyclone,before,2019-10-15,power outage,1183894014573105152,News_1jl4,California’s power outage means problems for e...,2019-10-14 23:55:03+00:00,NaN,NaN
3,ne_bomb_cyclone,before,2019-10-15,power outage,1183893791415123968,IndeCardio,Newsome is vanguard Globalism in action! What ...,2019-10-14 23:54:10+00:00,NaN,NaN
4,ne_bomb_cyclone,before,2019-10-15,power outage,1183893732652810240,BaddictsPH,Super Typhoon Faxai hit Chiba prefecture in Se...,2019-10-14 23:53:56+00:00,NaN,NaN


##### 2.1 Checking the DataFrame basic Format and columns

In [7]:
# print out the  size of the data and the number of rows and columns
print(df.head())
print(df.shape)
print(df.describe())

             Event   Stage  Query Date    Query Term                   Id  \
0  ne_bomb_cyclone  before  2019-10-15  power outage  1183894397492043777   
1  ne_bomb_cyclone  before  2019-10-15  power outage  1183894362725289984   
2  ne_bomb_cyclone  before  2019-10-15  power outage  1183894014573105152   
3  ne_bomb_cyclone  before  2019-10-15  power outage  1183893791415123968   
4  ne_bomb_cyclone  before  2019-10-15  power outage  1183893732652810240   

         Username                                               Text  \
0      meekers999  Let's lose Govenor Gruesome in Cali please. He...   
1  sharethiscrime  Last time I checked, he's still a floofy baby ...   
2       News_1jl4  California’s power outage means problems for e...   
3      IndeCardio  Newsome is vanguard Globalism in action! What ...   
4      BaddictsPH  Super Typhoon Faxai hit Chiba prefecture in Se...   

                        Date Hashtags  Location  
0  2019-10-14 23:56:34+00:00      NaN       NaN  
1  2

##### 2.2. Determining Missing observations

In [8]:
df.isnull().sum()

Event            0
Stage            0
Query Date       0
Query Term       0
Id               0
Username         0
Text             0
Date             0
Hashtags      4806
Location      6000
dtype: int64

In [9]:
# Checking missing values `.isnull() will convert to the boolean True and False and return new DataFrame where null values
# are indicated by True and the .sum() will give us the total numbel of null values per column.
# checking the null values in the DF , according to Ben this is the fancy one
(df.isnull().sum() / df.shape[0]).sort_values(ascending=False)   # credit Ben shaver

Location      1.000
Hashtags      0.801
Date          0.000
Text          0.000
Username      0.000
Id            0.000
Query Term    0.000
Query Date    0.000
Stage         0.000
Event         0.000
dtype: float64

#####  2.3. Describing the summary statistics of the columns

In [10]:
df.head()

,Event,Stage,Query Date,Query Term,Id,Username,Text,Date,Hashtags,Location
0,ne_bomb_cyclone,before,2019-10-15,power outage,1183894397492043777,meekers999,Let's lose Govenor Gruesome in Cali please. He...,2019-10-14 23:56:34+00:00,NaN,NaN
1,ne_bomb_cyclone,before,2019-10-15,power outage,1183894362725289984,sharethiscrime,"Last time I checked, he's still a floofy baby ...",2019-10-14 23:56:26+00:00,NaN,NaN
2,ne_bomb_cyclone,before,2019-10-15,power outage,1183894014573105152,News_1jl4,California’s power outage means problems for e...,2019-10-14 23:55:03+00:00,NaN,NaN
3,ne_bomb_cyclone,before,2019-10-15,power outage,1183893791415123968,IndeCardio,Newsome is vanguard Globalism in action! What ...,2019-10-14 23:54:10+00:00,NaN,NaN
4,ne_bomb_cyclone,before,2019-10-15,power outage,1183893732652810240,BaddictsPH,Super Typhoon Faxai hit Chiba prefecture in Se...,2019-10-14 23:53:56+00:00,NaN,NaN


In [11]:
# the describe function gives summary statistics for each variable
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,6000.0,1.184496e+18,7.203922e+14,1.183428e+18,1.183793e+18,1.184462e+18,1.185247e+18,1.185705e+18
Location,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# sns.distplot(df, kde = True)

##### 2.4.  Feature Engineering/ Exploring the Target Variable

In [13]:
#checking the number of unique titles in the data set
len(df['Text'].unique())

5442

In [14]:
x = list(df['Text'].head(3))   # calling the five titles as list from the Df
x

['Let\'s lose Govenor Gruesome in Cali please. He just ordered the shut down of all mountain top ham radio repeaters. That means "no emergency communications" will be available during say.. a power outage ands mass fires. He\'s trying to kill you.',
 "Last time I checked, he's still a floofy baby complaining about power outage ",
 'California’s power outage means problems for electric cars.. Washingtonpost - Twitter - News - Noticias - Bitcoin - CryptoCurrency - @InvestCrypForex - @1jl4com - @Health14Fitness - @Marketing_1jl4 - @News_1jl4 &gt;']